In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"C:\Users\nisho\Documents\nishoak docs\Studies\Machine Learning\Coding_Part\02_Data_Preprocessing\03_titanic_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df = df.loc[ df.Embarked.notnull(), ['Survived', 'Pclass', 'Sex', 'Embarked']] #.loc is label based (we have to specify row and column labels)
df.shape

(889, 4)

In [7]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [8]:
X = df.loc[ : , ['Pclass']]
y = df.Survived
print(X.shape)
print(y.shape)

(889, 1)
(889,)


In [9]:
X

,Pclass
0,3
1,1
2,3
3,1
4,3
...,...
886,2
887,1
888,3
889,1


In [10]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 889, dtype: int64

In [11]:
from sklearn.linear_model import LogisticRegression
lr_2 = LogisticRegression()

### Cross Validating a Model

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr_2, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

### One Hot Encoding

In [13]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [14]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [15]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [16]:
X = df.drop('Survived' , axis= 'columns')
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


### Cross validating a pipeline with all features

In [17]:
X = df.drop('Survived', axis='columns')
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [18]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer ( (OneHotEncoder(), ['Sex', 'Embarked']),remainder='passthrough' )

In [19]:

column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [20]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(column_trans, lr_2)

In [21]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

### Making predictions on new data

In [22]:
X_new = X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [23]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [24]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)

### Entire code at a glance

In [25]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [26]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]
X = df.drop('Survived', axis='columns')
y = df.Survived

In [27]:
column_trans = make_column_transformer ( (OneHotEncoder(), ['Sex', 'Embarked']), remainder='passthrough' )
lr_2 = LogisticRegression(solver='lbfgs')

In [28]:
pipe = make_pipeline(column_trans, lr_2)

In [29]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [30]:
X_new = X.sample(5, random_state=99)

In [31]:
pipe.fit(X, y)
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)